## Reading Data from Google Sheets

Let us see how we can read actual data from a Google Sheet.
* We have to create resource using `values` on top of `service.spreadsheets`.
* Then we need to send http request (invoke get) by passing both spreadsheet id as well as range of values.

> Here is how I am able to define the range for all rows and all 7 columns in my data set using the sheet name with in the main sheet.

In [ ]:
%run 05_overview_of_google_sheets_api.ipynb

In [ ]:
SPREADSHEET_ID = '1lgyVuw6nVyRnmKtCPbXF4kYcop5HMJ8H3eeNsArAlVk'

In [ ]:
def get_credentials():
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)

    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
            
    return creds

In [ ]:
creds = get_credentials()

In [ ]:
RANGE_NAME = 'Form Masked!A1:G'

In [ ]:
service = build('sheets', 'v4', credentials=creds)

In [ ]:
sheet = service.spreadsheets()

In [ ]:
sheet_values = sheet.values()

In [ ]:
type(sheet_values)

In [ ]:
sheet_details = sheet_values.get(spreadsheetId=SPREADSHEET_ID,
                            range=RANGE_NAME).execute()

In [ ]:
type(sheet_details)

In [ ]:
sheet_details

In [ ]:
sheet_details.keys()

In [ ]:
sheet_details['values'][0]

In [ ]:
sheet_details.get('values')[0]

In [ ]:
sheet_data = sheet_details.get('values', [])

In [ ]:
if not sheet_data:
    print('No data found.')
else:
    columns = sheet_data[0]
    rows = sheet_data[1:]

In [ ]:
for column in columns: print(column)

In [ ]:
rows[:3]

In [ ]:
def get_sheet_data(service, spreadsheet_id, spreadsheet_range):
    sheet = service.spreadsheets()
    sheet_values = sheet.values()
    sheet_details = sheet_values.get(spreadsheetId=spreadsheet_id,
                            range=spreadsheet_range).execute()
    return sheet_details.get('values')[0], sheet_details.get('values')[1:]

In [ ]:
get_sheet_data(service, SPREADSHEET_ID, RANGE_NAME)[0]

In [ ]:
get_sheet_data(service, SPREADSHEET_ID, RANGE_NAME)[1][0]

In [ ]:
columns, rows = get_sheet_data(service, SPREADSHEET_ID, RANGE_NAME)

In [ ]:
emails = [row[2] for row in rows]

In [ ]:
emails[:3]

In [ ]:
import pandas as pd

df = pd.DataFrame(rows, columns=columns)

In [ ]:
df.head(5)

In [ ]:
df['Email Address'].head(5)

In [ ]:
df.columns

In [ ]:
df.drop?

In [ ]:
df = df.drop(columns=df.columns[1]).drop(columns=df.columns[5:])

In [ ]:
df.head(5)

In [ ]:
df.columns=['submitted_ts', 'email_id', 'first_name', 'last_name']

In [ ]:
df.head(5)